In [1]:
import qubovert as qv
from qubovert import boolean_var

In [8]:
num = qv.integer_var('x', 8, True)
m = qv.PCBO()
m -= num
m

{('x0',): -1,
 ('x1',): -2,
 ('x2',): -4,
 ('x3',): -8,
 ('x4',): -16,
 ('x5',): -32,
 ('x6',): -64,
 ('x7',): -128}

In [29]:
import pickle
with open('pickles/50-322-4-7-sum_taus-2-3-113261-H.pickle', 'rb') as f:
    h = pickle.load(f)

import qubovert as qv
from qubovert import boolean_var

h_new = qv.PCBO()
objective_lambda = 0.1

def build_term_from_constraint(constraint):
    term = qv.PUBO()
    for pair_bools, coefficient in constraint.items():
        if len(pair_bools) == 0:
            term += coefficient
        elif len(pair_bools) == 1:
            term += boolean_var(pair_bools[0]) * coefficient
        else:
            term += boolean_var(pair_bools[0]) * boolean_var(pair_bools[1]) * coefficient
    return term

# ----- Add less than constraints and objective -----
for constraint in h['constraints']['lt']:
    left_term = build_term_from_constraint(constraint)
    h_new.add_constraint_lt_zero(left_term)
    # ---- In this problem, the objective is the same as 0.1 * h['constraints']['lt']
    h_new += objective_lambda * left_term

# ----- Add equal zero constraints -----
for constraint in h['constraints']['eq']:
    left_term = build_term_from_constraint(constraint)
    h_new.add_constraint_eq_zero(left_term)
assert h_new.to_qubo().Q == h['qubo']

In [15]:
h['constraints']['lt'][0]#[('tau_13',)]

{('tau_13',): 1,
 ('tau_10',): 1,
 ('tau_7',): 1,
 ('tau_11',): 1,
 ('tau_6',): 1,
 ('tau_9',): 1,
 ('tau_14',): 1,
 (): -7}

In [10]:
h['qubo']

{(0, 0): -11.9,
 (1, 1): -9.9,
 (2, 2): -9.9,
 (3, 3): -13.9,
 (4, 4): -7.9,
 (5, 5): -11.9,
 (6, 6): -13.9,
 (0, 1): 2,
 (0, 2): 2,
 (0, 3): 2,
 (0, 4): 2,
 (0, 5): 2,
 (0, 6): 2,
 (0, 7): 2,
 (0, 8): 4,
 (0, 9): 8,
 (1, 2): 2,
 (1, 3): 2,
 (1, 4): 2,
 (1, 5): 2,
 (1, 6): 2,
 (1, 7): 2,
 (1, 8): 4,
 (1, 9): 8,
 (2, 3): 2,
 (2, 4): 2,
 (2, 5): 2,
 (2, 6): 2,
 (2, 7): 2,
 (2, 8): 4,
 (2, 9): 8,
 (3, 4): 2,
 (3, 5): 2,
 (3, 6): 2,
 (3, 7): 2,
 (3, 8): 4,
 (3, 9): 8,
 (4, 5): 2,
 (4, 6): 2,
 (4, 7): 2,
 (4, 8): 4,
 (4, 9): 8,
 (5, 6): 2,
 (5, 7): 2,
 (5, 8): 4,
 (5, 9): 8,
 (6, 7): 2,
 (6, 8): 4,
 (6, 9): 8,
 (7, 7): -11,
 (8, 8): -20,
 (9, 9): -32,
 (7, 8): 4,
 (7, 9): 8,
 (8, 9): 16,
 (10, 10): 32.0,
 (4, 10): -2,
 (11, 11): 32.0,
 (2, 11): -2,
 (5, 12): 2,
 (12, 12): 30.0,
 (1, 13): 2,
 (13, 13): 30.0,
 (3, 14): 2,
 (14, 14): 30.0,
 (0, 15): 2,
 (15, 15): 30.0,
 (6, 16): 2,
 (16, 16): 30.0,
 (17, 17): -29.0,
 (17, 18): 4,
 (17, 19): 8,
 (17, 20): 16,
 (17, 21): 32,
 (10, 17): -2,
 (11,

In [2]:
from neal import SimulatedAnnealingSampler
def solve(H):
    H_qubo = H.to_qubo().Q
    sim_sampler = SimulatedAnnealingSampler()
    samples = sim_sampler.sample_qubo(H_qubo, num_reads=NUM_READS)
    return samples

In [3]:
def H(v):
    H = qv.PCBO()
    for k in range(len(taus)):
        H += h[k] * taus[k]
    
    for i in range(len(F)):
        H += - v[i] * F[i]
    samples = solve(H)
    f = []
    for i in range(len(F)):
        exp_val_f_i = 0
        for sample in samples:
            H_solution = H.convert_solution(sample)
            f_i_val = F[i].value(H_solution)
            exp_val_f_i += f_i_val
        H_solution = H.convert_solution(samples.first.sample)
        exp_val_f_i = exp_val_f_i / len(samples)
        f.append(exp_val_f_i)
    return [f,H_solution]

In [5]:
import random
from numpy import sqrt, abs
NUM_READS = 100
num_epochs = 100
log_epochs = num_epochs/100 # Total number of log lines

N = 80
taus = {i: boolean_var(f't_{i}') for i in range(N)}

F0 = sum(taus.values()) - 7
F1 =  boolean_var('t_0') + boolean_var('t_1') + boolean_var('t_2') - 3
F2 =  boolean_var('t_3') + boolean_var('t_4') + boolean_var('t_5') - 0

F = [F0, F1, F2]


H_orig = qv.PCBO()
for i in range(len(F)):
    H_orig.add_constraint_eq_zero(F[i])

v = {i: random.random() for i in range(len(F))}
h = {i: random.random()/10 for i in range(len(taus))}
print(f'initial v: {v}')

#####ADAM Optimizer#######
Alpha = 0.1
Beta1 = 0.9
Beta2 = 0.999
m = [0] * len(F)
n = [0] * len(F)
t = 0
#####ADAM Optimizer#######
ExpF = 1.0

while ExpF != [0.0] * len(F):
    epoch = t
    
    #####ADAM Optimizer#######
    t = t + 1
    [ExpF, Solution] = H(v)
    for i in range(len(F)): 
        m[i] = Beta1 * m[i] + (1-Beta1)*ExpF[i]
        n[i] = Beta2 * n[i] + (1-Beta2)*(ExpF[i]**2)
        me = m[i]/(1-Beta1**t)
        ne = n[i]/(1-Beta2**t)
        v[i] = v[i] - Alpha * (me/(sqrt(ne)+0.00000001))
    #####ADAM Optimizer#######
    
        if epoch != 0 and epoch % log_epochs == 0:
            print(f'epoch {epoch} v[{i}]: {v[i]}, \t Expectation Value of F{i}: {ExpF[i]} \t Constraint Error: {F[i].value(Solution)}')
    if epoch != 0 and epoch % log_epochs == 0:
        print(f'Gradient: {ExpF}')
        print(f'Energy value ')
        print()

print('is_solution_valid:', H_orig.is_solution_valid(Solution))
sorted(Solution.items(), key=lambda item: int(item[0][2:]))

initial v: {0: 0.4491684829937874, 1: 0.9474242849590829, 2: 0.7180160191031555}
epoch 1 v[0]: 0.2491688403441495, 	 Expectation Value of F0: 72.99 	 Constraint Error: 73
epoch 1 v[1]: 0.9474242849590829, 	 Expectation Value of F1: 0.0 	 Constraint Error: 0
epoch 1 v[2]: 0.5180160197698228, 	 Expectation Value of F2: 3.0 	 Constraint Error: 3
Gradient: [72.99, 0.0, 3.0]
Energy value 

epoch 2 v[0]: 0.14916882371764123, 	 Expectation Value of F0: 73.0 	 Constraint Error: 73
epoch 2 v[1]: 0.9474242849590829, 	 Expectation Value of F1: 0.0 	 Constraint Error: 0
epoch 2 v[2]: 0.4180160201031562, 	 Expectation Value of F2: 3.0 	 Constraint Error: 3
Gradient: [73.0, 0.0, 3.0]
Energy value 

epoch 3 v[0]: 0.04916862744532871, 	 Expectation Value of F0: 73.0 	 Constraint Error: 73
epoch 3 v[1]: 0.9474242849590829, 	 Expectation Value of F1: 0.0 	 Constraint Error: 0
epoch 3 v[2]: 0.31801602043649, 	 Expectation Value of F2: 3.0 	 Constraint Error: 3
Gradient: [73.0, 0.0, 3.0]
Energy value 

ep

[('t_0', 1),
 ('t_1', 1),
 ('t_2', 1),
 ('t_3', 0),
 ('t_4', 0),
 ('t_5', 0),
 ('t_6', 0),
 ('t_7', 0),
 ('t_8', 0),
 ('t_9', 0),
 ('t_10', 0),
 ('t_11', 0),
 ('t_12', 0),
 ('t_13', 0),
 ('t_14', 0),
 ('t_15', 0),
 ('t_16', 0),
 ('t_17', 0),
 ('t_18', 0),
 ('t_19', 0),
 ('t_20', 0),
 ('t_21', 0),
 ('t_22', 0),
 ('t_23', 0),
 ('t_24', 0),
 ('t_25', 0),
 ('t_26', 0),
 ('t_27', 0),
 ('t_28', 0),
 ('t_29', 0),
 ('t_30', 0),
 ('t_31', 0),
 ('t_32', 0),
 ('t_33', 0),
 ('t_34', 0),
 ('t_35', 0),
 ('t_36', 1),
 ('t_37', 0),
 ('t_38', 0),
 ('t_39', 0),
 ('t_40', 0),
 ('t_41', 0),
 ('t_42', 0),
 ('t_43', 0),
 ('t_44', 0),
 ('t_45', 0),
 ('t_46', 0),
 ('t_47', 0),
 ('t_48', 0),
 ('t_49', 0),
 ('t_50', 0),
 ('t_51', 0),
 ('t_52', 0),
 ('t_53', 0),
 ('t_54', 0),
 ('t_55', 0),
 ('t_56', 0),
 ('t_57', 0),
 ('t_58', 1),
 ('t_59', 0),
 ('t_60', 0),
 ('t_61', 1),
 ('t_62', 0),
 ('t_63', 0),
 ('t_64', 0),
 ('t_65', 0),
 ('t_66', 0),
 ('t_67', 0),
 ('t_68', 0),
 ('t_69', 0),
 ('t_70', 1),
 ('t_71', 0),
 (

In [8]:
H = qv.PCBO()
for k in range(len(taus)):
    H += h[k] * taus[k]

for i in range(len(F)):
    H += - v[i] * F[i]
samples = solve(H)
samples;

In [9]:
for sample in samples:
    cost = 0
    for i in range(len(sample)):
        cost += sample[i]*h[i]
#     print(cost)